## KDT 지원자 통계 프로그램 사용법 안내
* 지원자의 분류에 따라 시각화하는 프로그램입니다.
1. 아래의 ▷ 버튼(코드 표시 버튼 왼쪽)을 클릭합니다
1. 기존 지원자 정보가 들어있는 xlsx 파일을 업로드합니다.
1. 자동으로 시각화 결과가 다운로드됩니다

In [ ]:
#@title KDT 지원자 통계 프로그램

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

from IPython.display import clear_output
from google.colab import files
import pandas as pd
from requests import get  # to make GET request
from google.colab import files

def download(url, file_name):
    with open(file_name, "wb") as file:   # open in binary mode
        response = get(url)               # get request
        file.write(response.content)      # write to file

# 대학정보 다운로드
download("https://docs.google.com/uc?export=download&id=16yzLYwCPcx6MgutXBCqSoMy3AJsxZUKR", "전국대학및전문대학정보표준데이터.csv")
download("https://docs.google.com/uc?export=download&id=1wWqIGC0vy7az0wJMUVZMQtTcQub6E5Oe", "양식.xlsx")

import re, os, time, shutil, io
import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

width = 72
height = 22

ctime = datetime.datetime.now()


def find_school(x):
    if re.search('[가-힣 ]+대학원', x):
        return re.search('[가-힣 ]+대학원', x).group()

    elif re.search('[가-힣 ]+캠퍼스', x):
        return re.search('[가-힣 ]+캠퍼스', x).group()

    elif re.search('[가-힣 ]+대학교', x):
        return re.search('[가-힣 ]+대학교', x).group()

    elif re.search('[가-힣 ]+대학', x):
        return re.search('[가-힣 ]+대학', x).group()

    elif re.search('[가-힣 ]+고등학교', x):
        return re.search('[가-힣 ]+고등학교', x).group()

    elif re.search('[가-힣 ]+전문학교', x):
        return re.search('[가-힣 ]+전문학교', x).group()

    elif '학점은행' in  x:
        return '학점은행'

    elif '국가평생교육진흥원' in x:
        return '국가평생교육진흥원'

    else:
        return x

def find_dept(x):
    if re.search('[a-zA-Z가-힣,.]+학부', x):
        return re.findall('[a-zA-Z가-힣,.]+학부', x)[-1]

    elif re.search('[a-zA-Z가-힣,. ]+공학', x):
        return re.findall('[a-zA-Z가-힣,. ]+공학', x)[-1]

    elif re.search('[a-zA-Z가-힣,.]+전공', x):
        return re.findall('[a-zA-Z가-힣,.]+전공', x)[-1]

    elif re.search('[a-zA-Z가-힣,.]+과', x):
        return re.findall('[a-zA-Z가-힣,.]+과', x)[-1]

    elif re.search('\([a-zA-Z가-힣,.]+\)', x):
        return re.findall('\([a-zA-Z가-힣,.]+\)', x)[-1][1:-1]

    elif ' ' not in x:
        return float('nan')

    else:
        return find_dept(x.replace(' ', ''))

def agechk(x):
    try:
        birth = str(x)

        if len(birth) != 6:
            raise

        age = int((ctime - datetime.datetime.strptime(birth,'%y%m%d')).days / 365)
        return age
    except:
        return None

IT_컴퓨터 = ['IT_컴퓨터', '정보', '통신', 'IT', '데이터', '컴퓨터', '소프트', 'AI']
교육 = ['교육', '사범', '교직']
수리통계 = ['수리통계', '수리', '수학', '통계']
사회_경제 = ['사회_경제', '경영', '경제', '행정', '사회', '복지', '기업']
공학 = ['공학', '전기', '산업', '전자', '반도체', '소재', '재료', '제어', '계측', '기계', '항공']
미디어 = ['미디어', '영상', '컨텐츠', '그래픽']
어문학 = ['어문학', '언어', '문학', '어문', '어학', '심리']
자연과학 = ['자연과학', '화학', '생물', '지구', '바이오', '의료', '물리', '간호', '의예', '식품']
예체능 = ['예체능', '예술', '체육', '음악', '미술', '성악', '디자인']

def dept_trim(x, ind):
    dept_list = [IT_컴퓨터, 교육, 수리통계, 사회_경제, 공학, 미디어, 어문학, 자연과학, 예체능]

    if type(x) == str:
        x = x.replace(' ', '')
        x = x.upper()

        for c in dept_list:
            for c2 in c:
                if c2 in x:
                    return c[0]

    else:
        return '기타'

# 연령
def age_calc2(x):
    if not x:
        return ''

    elif x < 20:
        return '~19'

    elif x >= 35:
        return '35+'

    else:
        return str(x)

def add_age():
    global ws, df_age
    i = 25

    for r in dataframe_to_rows(df_age, index=False, header=False):
        ws.cell(i, 1, r[0])
        ws.cell(i, 2, r[1])
        i += 1


# 대학교 정보, 교육생 자료
univ = pd.read_csv('전국대학및전문대학정보표준데이터.csv', encoding='cp949')

clear_output(wait=True)

print('지원자 정보 엑셀파일을 업로드해 주세요')
answer = files.upload()
for k in answer.keys():
    wb_orig = openpyxl.load_workbook(io.BytesIO(answer.get(k)))
sheets = wb_orig.get_sheet_names()


# KDT 교육생 인원수 카운트
all_num = 0

for sheet in sheets:
    df = pd.read_excel(io.BytesIO(answer.get(k)), sheet_name=sheet)
    globals()[df.loc[0, '과정'].replace(' ', '')] = df.shape[0]

    all_num += globals()[df.loc[0, '과정'].replace(' ', '')]


for sheet in sheets:
    df = pd.read_excel(io.BytesIO(answer.get(k)), sheet_name=sheet)

    df['학교'] = df['최종학력'].apply(lambda x: find_school(x))
    df['학과'] = df['최종학력'].apply(lambda x: find_dept(x))
    df['나이'] = df['생년월일'].apply(lambda x: agechk(x))

    df['학교2'] = df['학교'].apply(lambda x: x.split(' ')[0])
    df['캠퍼스'] = df['학교'].apply(lambda x: re.search('[가-힣]+캠퍼스', x).group()[:-3] if '캠퍼스' in x else '본교')
    df['학과'] = [dept_trim(x, 0) for x in df['학과']]
    df['나이'] = df['나이'].fillna(-1).astype('int').replace({-1: None})

    df_gender = df.value_counts('성별')

    plt.pie(df_gender, autopct='%.1f')
    plt.title('교육생 성비')
    plt.legend(df_gender.index, loc='right', bbox_to_anchor=(1.2, 0.5))
    plt.savefig('교육생 성비.png', bbox_inches='tight')
    plt.clf()

    df_age = df.value_counts('나이').sort_index()

    plt.pie(df_age, autopct='%.1f', pctdistance=.8, labels=[str(x)+'세' for x in df_age.index], rotatelabels=False, labeldistance=1.1)
    plt.title('교육생 연령')
    plt.legend(df_age.index, loc='right', bbox_to_anchor=(1.3, 0.5))
    plt.savefig('교육생 연령.png', bbox_inches='tight')
    plt.clf()

    temp = df['나이'].apply(lambda x: str(x)[0] + '0대' if x else x)
    df_age2 = temp.value_counts().sort_index()

    plt.pie(df_age2, autopct='%.1f')
    plt.title('교육생 연령대')
    plt.legend(df_age2.index, loc='right', bbox_to_anchor=(1.2, 0.5))
    plt.savefig('교육생 연령대.png', bbox_inches='tight')
    plt.clf()

    df_dept = df.value_counts('학과').sort_index()

    plt.pie(df_dept, autopct='%.1f', pctdistance=.7, labels=df_dept.index, rotatelabels=False, labeldistance=1.1)
    plt.title('교육생 전공')
    plt.legend(df_dept.index, loc='right', bbox_to_anchor=(1.5, 0.5))
    plt.savefig('교육생 전공.png', bbox_inches='tight')
    plt.clf()

    df_school = df.value_counts('학교2').sort_index()

    # plt.pie(df_school, autopct='%.1f', pctdistance=1.15)
    # plt.title('교육생 출신 학교')
    # plt.legend(df_school.index, loc='right', bbox_to_anchor=(1.7, 0.5))
    # plt.savefig('교육생 학교.png', bbox_inches='tight')
    # plt.clf()

    df['학교2'] = df['학교2'] + df['캠퍼스']
    univ2 = univ[['학교명','본분교구분명','시도명']]

    univ2['캠퍼스명'] = univ2['학교명'].apply(lambda x: re.findall('\([a-zA-Z가-힣,. ]+\)', x)[-1][1:-1] if re.search('\([a-zA-Z가-힣,. ]+\)', x) else '')
    univ2['학교명'] = univ2['학교명'].apply(lambda x: x.split('(')[0].split(' ')[0])

    univ2.drop_duplicates(inplace=True)
    univ2['본분교구분명'] = univ2['본분교구분명'].apply(lambda x: x+'스' if x.endswith('캠퍼') else x)

    for i in univ2.index:
        if univ2.loc[i, '본분교구분명'] == '본교':
            univ2.loc[i, '학교명'] += '본교'
        elif univ2.loc[i, '캠퍼스명']:
            univ2.loc[i, '학교명'] = univ2.loc[i, '학교명'] + univ2.loc[i, '캠퍼스명']

    univ2.drop_duplicates('학교명', inplace=True)

    dfs = pd.merge(df, univ2, left_on='학교2', right_on='학교명', how='left')

    df_school2 = dfs.value_counts('시도명').sort_index()

    plt.pie(df_school2, autopct='%.1f', pctdistance=.8, labels=df_school2.index, rotatelabels=True, labeldistance=1.1)
    plt.title('교육생 지역\n\n\n')
    plt.legend(df_school2.index, loc='right', bbox_to_anchor=(1.6, 0.5))
    plt.savefig('교육생 지역.png', bbox_inches='tight')
    plt.clf()

    df_gender = pd.DataFrame(df_gender, columns=['인원수']).reset_index()

    df_age = pd.DataFrame(df_age, columns=['인원수']).reset_index()

    df_age2 = pd.DataFrame(df_age2)
    df_age2.columns = ['인원수']
    df_age2.reset_index(inplace=True)

    df_dept = pd.DataFrame(df_dept, columns=['인원수']).reset_index()

    df_school = pd.DataFrame(df_school).reset_index()
    df_school.columns = ['학교','인원수']

    df_school2 = pd.DataFrame(df_school2, columns=['인원수']).reset_index()

    title = df.loc[0,'과정']

    if os.path.isfile('./kdt_temp.xlsx'):
        wb = openpyxl.load_workbook('./kdt_temp.xlsx')
    else:
        wb = openpyxl.load_workbook('./양식.xlsx')

    ws = wb.active
    ws = wb.get_sheet_by_name(title)

    ws['C3'] = all_num
    ws['C4'] = 빅데이터
    ws['C5'] = Java풀스택
    ws['C6'] = PM

    # 성별
    ws['B14'] = df_gender.loc[0, '인원수']
    ws['B15'] = df_gender.loc[1, '인원수']

    img = Image('./교육생 성비.png')
    img.width = width*4
    img.height = height*11
    ws.add_image(img, 'D12')

    if df_age.shape[0] > 15:
        df['나이'] = df['나이'].apply(lambda x: age_calc2(x))
        df_age = df.value_counts('나이').sort_index()
        df_age = pd.DataFrame(df_age, columns=['인원수']).reset_index()

    add_age()

    img = Image("./교육생 연령대.png")
    img.width = width*4
    img.height = height*12
    ws.add_image(img, 'D24')

    # 지역
    i = 44

    for r in dataframe_to_rows(df_school2, index=False, header=False):
        ws.cell(i, 1, r[0])
        ws.cell(i, 2, r[1])
        i += 1

    img = Image("./교육생 지역.png")
    img.width = width*6
    img.height = height*17
    ws.add_image(img, 'C42')

    # 전공
    i = 63

    for r in dataframe_to_rows(df_dept, index=False, header=False):
        ws.cell(i, 2, r[1])
        i += 1

    img = Image("./교육생 전공.png")
    img.width = width*6
    img.height = height*14
    ws.add_image(img, 'C61')

    # 학교
    if df_school.shape[0] > 39:
        i = 84

        for r in dataframe_to_rows(df_school.iloc[:int(df_school.shape[0] / 2)], index=False, header=True):
            ws.cell(i, 1, r[0])
            ws.cell(i, 2, r[1])
            i += 1

        i = 84
        for r in dataframe_to_rows(df_school.iloc[int(df_school.shape[0] / 2) + 1:], index=False, header=True):
            ws.cell(i, 3, r[0])
            ws.cell(i, 4, r[1])
            i += 1

    else:
        i = 84

        for r in dataframe_to_rows(df_school, index=False, header=True):
            ws.cell(i, 1, r[0])
            ws.cell(i, 2, r[1])
            i += 1

    wb.save("kdt_temp.xlsx")

shutil.copy("./kdt_temp.xlsx", "./kdt_result.xlsx")
os.remove("kdt_temp.xlsx")

clear_output(wait=True)
files.download('./kdt_result.xlsx')
print("파일이 자동으로 다운로드됩니다")